# Deploying a Serverless Endpoint
***

This notebook is for  creating a serverless endpoint and deploying a model created used with slf-classifier. Both are done with the same command.

Code is also included to test the endpoint.

In [9]:
import sagemaker, boto3
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from sagemaker.session import Session



# model_version="*" fetches the latest version of the model.
infer_model_id, infer_model_version = "tensorflow-ic-imagenet-mobilenet-v2-100-224-classification-4", "*"
endpoint_name = name_from_base(f"slf-serverless")

inference_instance_type = "ml.m5.xlarge"

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()

In [11]:
# Retrieve the inference docker container uri.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=infer_model_id,
    model_version=infer_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri.
deploy_source_uri = script_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, script_scope="inference"
)

# Assemble the URI to the model we have trained for slf
input_bucket = bucket
input_prefix = "slf-classifier-training/output/slf-tensorflow-ic-imagenet-mobilenet-v2-2023-01-25-03-06-14-286/output" # Not sure how to autogen
input_model_name = "model.tar.gz"
base_model_uri = f"s3://{input_bucket}/{input_prefix}/{input_model_name}"


# Create the SageMaker model instance. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=base_model_uri,
    entry_point="inference.py",
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name + "-model",
)

In [12]:
from sagemaker.serverless import ServerlessInferenceConfig
serverless_config = ServerlessInferenceConfig(memory_size_in_mb=1024, max_concurrency=3)

In [13]:
# deploy the Model.
model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
    serverless_inference_config=serverless_config
)

---------!

# Testing the Endpoint
***

In [10]:
s3_bucket = f"lantern-rd-east2"


def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{image_key}", filename)


slf_images = {
    "test1.jpg": "slf/0.jpg",
    "test2.jpg": "slf/347.jpg",
}
download_from_s3(slf_images)

notSlf_images = {
    "test11.jpg": "not-slf/0.jpg",
    "test12.jpg": "not-slf/347.jpg",
}


images = [slf_images, notSlf_images]

In [ ]:
download_from_s3(notSlf_images)

In [11]:
from IPython.core.display import HTML
import json

for testSet in images:
    for image_filename in testSet.keys():
        with open(image_filename, "rb") as file:
            img = file.read()
        query_response = model_predictor.predict(
            img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
        )
        model_predictions = json.loads(query_response)
        predicted_label = model_predictions["predicted_label"]
        display(
            HTML(
                f'<img src={image_filename} alt={testSet[image_filename]} align="left" style="width: 250px;"/>'
                f"<figcaption>Image Name: {image_filename}</figcaption>"
                f"<figcaption>Predicted Label: {predicted_label}</figcaption>"
            )
        )

NameError: name 'model_predictor' is not defined